In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
# Paso 1: Preprocesamiento de datos
data_taxi = pd.read_csv(
    "data_taxi.csv"
)  # Reemplaza 'data_taxi.csv' con la ubicación correcta del archivo
data_taxi = data_taxi.dropna()  # Eliminar filas con valores faltantes o NaN

In [ ]:
# Agregar la extracción de horas y dias de la semana

In [ ]:
# Paso 2: Dividir los datos en características (X) y etiquetas (y)
X = data_taxi[["VendorID"]]  # Características
y = data_taxi[["duration"]]  # Etiquetas

In [ ]:
# Paso 3: Codificar las etiquetas
label_encoder = LabelEncoder()
y_encoded = y.apply(label_encoder.fit_transform)

In [ ]:
# Paso 4: Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

In [ ]:
# Paso 5: Normalizar los datos
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Cargar el dataframe 'data_taxi'
data_taxi = pd.read_csv(
    "ruta_del_archivo.csv"
)  # Asegúrate de reemplazar 'ruta_del_archivo.csv' con la ruta correcta de tu archivo

# Dividir el dataframe en características (X) y objetivos (y) para el primer modelo
X_all = data_taxi.drop(["fare_amount", "duration"], axis=1)
y_all = data_taxi[["fare_amount", "duration"]]

# Normalizar los datos de características para el primer modelo
scaler_all = MinMaxScaler()
X_all_normalized = scaler_all.fit_transform(X_all)

# Dividir los datos en conjuntos de entrenamiento y prueba para el primer modelo
X_all_train, X_all_test, y_all_train, y_all_test = train_test_split(
    X_all_normalized, y_all, test_size=0.2, random_state=42
)

# Crear el primer modelo de red neuronal para todas las características
model_all = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(
            64, activation="relu", input_shape=(X_all_train.shape[1],)
        ),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(
            2
        ),  # 2 neuronas de salida para 'fare_amount' y 'duration'
    ]
)

# Compilar y entrenar el primer modelo
model_all.compile(optimizer="adam", loss="mse")
model_all.fit(
    X_all_train,
    y_all_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_all_test, y_all_test),
)

# Crear el segundo modelo sin las variables 'PULocationID' y 'DOLocationID'
X_partial = data_taxi.drop(
    ["PULocationID", "DOLocationID", "fare_amount", "duration"], axis=1
)
y_partial = data_taxi[["fare_amount", "duration"]]

# Normalizar los datos de características para el segundo modelo
scaler_partial = MinMaxScaler()
X_partial_normalized = scaler_partial.fit_transform(X_partial)

# Dividir los datos en conjuntos de entrenamiento y prueba para el segundo modelo
X_partial_train, X_partial_test, y_partial_train, y_partial_test = train_test_split(
    X_partial_normalized, y_partial, test_size=0.2, random_state=42
)

# Crear el segundo modelo de red neuronal sin las variables 'PULocationID' y 'DOLocationID'
model_partial = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(
            64, activation="relu", input_shape=(X_partial_train.shape[1],)
        ),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(
            2
        ),  # 2 neuronas de salida para 'fare_amount' y 'duration'
    ]
)

# Compilar y entrenar el segundo modelo
model_partial.compile(optimizer="adam", loss="mse")
model_partial.fit(
    X_partial_train,
    y_partial_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_partial_test, y_partial_test),
)

# Solicitar al usuario las zonas de pickup y dropoff
pickup_zone = int(input("Ingrese la zona de recogida (PULocationID): "))
dropoff_zone = int(input("Ingrese la zona de entrega (DOLocationID): "))

# Preparar los datos para la predicción
input_data = pd.DataFrame(
    [[pickup_zone, dropoff_zone]], columns=["PULocationID", "DOLocationID"]
)
input_data_normalized_all = scaler_all.transform(input_data)
input_data_normalized_partial = scaler_partial.transform(
    input_data.drop(["PULocationID", "DOLocationID"], axis=1)
)

# Realizar la predicción de 'fare_amount' y 'duration' utilizando el primer modelo
prediction_all = model_all.predict(input_data_normalized_all)

# Realizar la predicción de 'fare_amount' y 'duration' utilizando el segundo modelo
prediction_partial = model_partial.predict(input_data_normalized_partial)

print("Prediction (using all features):")
print("fare_amount:", prediction_all[0][0])
print("duration:", prediction_all[0][1])

print("Prediction (excluding PULocationID and DOLocationID):")
print("fare_amount:", prediction_partial[0][0])
print("duration:", prediction_partial[0][1])